- __ResNet50 is a well-known model, which is commonly reused as a stock model, such as for `transfer learning`, as shared layers in object detection, and for performance benchmarking. The model has three versions: v1, v1.5 and v2.__
- __RestNet50 v1 formalized the concept of a `convolutional group`. This is a set of convolutional blocks that share a common configuration, such as the number of filters. In `v1`, the neural network is decomposed into groups, and each group doubles the number of filters from the prevoius group.__

**The following is an implementation of ResNet50 v1 using the bottleneck block combined with `batch normalization`**

In [7]:
from tensorflow.keras import Model
import tensorflow.keras.layers as layers

In [11]:
def identity_block(n_filters, x):
    """ Create a Bottleneck Residual Block of Convolutions
        n_filters: number of filters.
        x: input to the block.
    """
    shortcut = x
    x = layers.Conv2D(n_filters, (1,1), strides=(1,1))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(n_filters, (3,3), strides=(1,1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(n_filters*4, (1,1), strides=(1,1))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.add([shortcut, x])
    x = layers.ReLU()(x)
    
    return x

In [12]:
def projection_block(x, n_filters, strides=(2,2)):
    """ Create Block of Convolutions with feature pooling
        Increase the number of filters by 4X.
        
        x: input into the block
        n_filters: number of the filters
    """
    # 1x1 projection convolution on shortcut to match size of output
    shortcut = layers.Conv2D(4*n_filters, (1,1), strides=(strides))(x)
    shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Conv2D(n_filters, (1,1), strides=strides)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(n_filters, (3,3), strides=(1,1), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(4*n_filters, (1,1), strides=(1,1))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    
    return x

In [13]:
inputs = layers.Input(shape=(224, 224, 3))

x = layers.ZeroPadding2D(padding=(3,3))(inputs)
x = layers.Conv2D(64, kernel_size=(7,7), strides=(2,2), padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.ZeroPadding2D(padding=(1,1))(x)
x = layers.MaxPool2D(pool_size=(3,3), strides=(2,2))(x)

# Each convolution group after the first group starts with a projection block.
x = projection_block(x, 64, strides=(1,1))
for _ in range(2):
    x = identity_block(64, x)
    
x = projection_block(x, 128)
for _ in range(3):
    x = identity_block(128, x)
    
x = projection_block(x, 256)
for _ in range(5):
    x = identity_block(256, x)
    
x = projection_block(x, 512)
for _ in range(2):
    x = identity_block(512, x)
    
x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(1000, activation='softmax')(x)
model = Model(inputs, outputs)

______________________________________________________________________________________________________________
______________________________________________________________________________________________________________

**The following is an implementation of ResNet50 v1 residual block with a `projection link`**

In [14]:
def projection_block(x, n_filters, strides=(2,2)):
    """ Create Block of Convolution with feature pooling
        Increase the number of filters by 4X.
        
        x: input into the block
        n_filters: number of filters
    """
    shortcut = layers.Conv2D(4*n_filters, (1,1), strides=(strides))(x)
    shortcut = layers.BatchNormalization()(shortcut)
    
    x = layers.Conv2D(n_filters, (1,1), strides=(1,1))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Bottleneck is moved to 3x3 convolution using a stride of 2
    x = layers.Conv2D(n_filters, (3,3), strides=(2,2), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.Conv2D(4*n_filters, (1,1), strides=(1,1))(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    
    return x

______________________________________________________________________________________________________________
______________________________________________________________________________________________________________

__ResNet50 v2 introduced `preactivation batch normalization` in which the batch normalization and activation functions are placed before (instead of after) the corresponding convolution or dense layer. This has now become a common practice, as depicted here for implementation of the residual block with the `identity link` in `v2`:__

In [15]:
def identity_block(x, n_filters):
    """ Create a Bottleneck Residual Block of Convolutions.
    
        n_filters: number of filters.
        x: input into the block.
    """
    shortcut = x
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(n_filters, (1,1), strides=(1,1))(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(n_filters, (3,3), strides=(1,1), padding='same')(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(n_filters*4, (1,1), strides=(1,1))(x)
    
    x = layers.add([shortcut, x])
    
    return x